In [3]:
"""

SINGLE notebook for merging all things related to clustering strokes, and stroke similarity metrics.

# NOTES: combines code from:

# ALSO SEE:
# devo_strokemanifold_analysis_040321, has some very old plots, and old notes.

"""



'\n\nSINGLE notebook for merging all things related to clustering strokes, and stroke similarity metrics.\n\n# NOTES: combines code from:\n\n# ALSO SEE:\n# devo_strokemanifold_analysis_040321, has some very old plots, and old notes.\n\n'

In [4]:
from pythonlib.tools.plottools import savefig
from pythonlib.dataset.dataset_analy.primitives import *
from pythonlib.dataset.dataset_preprocess.primitives import *
from pythonlib.dataset.dataset import Dataset, load_dataset_daily_helper
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [5]:


animal = "Pancho"
date = "221220"

# Diego, char with clust labels
# animal = "Diego"
# date = "231204"

rename_shapes_if_cluster_labels_exist = True
D = load_dataset_daily_helper(animal, date, rename_shapes_if_cluster_labels_exist=rename_shapes_if_cluster_labels_exist)


Searching using this string:
/home/lucast4/code/drawmonkey/expt_metadat/*221220-*Pancho.**
Found this many paths:
0
Searching using this string:
/home/lucast4/code/drawmonkey/expt_metadat_daily/*221220-*Pancho.**
Found this many paths:
1
---
/home/lucast4/code/drawmonkey/expt_metadat_daily/priminvar3j-221220-Pancho.yaml
Loading this dataset Pancho priminvar3j 221220
Searching using this string:
/gorilla1/analyses/database/*Pancho-*priminvar3j-*221220-*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
0
Searching using this string:
/gorilla1/analyses/database/BEH/*Pancho-*priminvar3j-*221220-*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/gorilla1/analyses/database/BEH/Pancho-priminvar3j-221220-221230_151621
Searching using this string:
/mnt/Freiwald_kgupta/kgupta/analyses/database/*Pancho-*priminvar3j-*221220-*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
0
Searching using this string:
/mnt/Freiwald_kgupta/kgupta/analyses/da


KeyboardInterrupt



Error in callback <function flush_figures at 0x7eff7b3318b0> (for post_execute):



KeyboardInterrupt



## Entire pipeline from character_cluster.py is below

In [ ]:
# WHICH_LEVEL = "shapemean"
WHICH_LEVEL = "trial"
WHICH_BASIS_SET = "Pancho"
# WHICH_BASIS_SET = "Diego"
WHICH_TASK_KIND = "character"
# WHICH_TASK_KIND = "prims_on_grid"
SUBSAMPLE = False

In [ ]:
# Keep only characters
D.Dat = D.Dat[D.Dat["task_kind"] == WHICH_TASK_KIND].reset_index(drop=True)

if SUBSAMPLE:
    # OPTIONAL
    D.subsampleTrials(1, 1)
from pythonlib.dataset.dataset_strokes import preprocess_dataset_to_datstrokes
from pythonlib.dataset.dataset_strokes import DatStrokes
DS = preprocess_dataset_to_datstrokes(D, "clean_chars")
SDIR = D.make_savedir_for_analysis_figures_BETTER(f"strokes_clustering_similarity/{WHICH_LEVEL}-basis_{WHICH_BASIS_SET}")
# SDIR = f"{SDIR}/{WHICH_LEVEL}"
# print(SDIR)

In [ ]:
WHICH_FEATURE = "beh_motor_sim" 

##### Perform clustering
ClustDict, ParamsDict, ParamsGeneral, dfdat = DS.features_wrapper_generate_all_features(WHICH_LEVEL, 
                                                                                        which_basis_set=WHICH_BASIS_SET)
plt.close("all")

# For each trial, extracting clustering score, etc.
DS.clustergood_assign_data_to_cluster(ClustDict, ParamsDict, 
            ParamsGeneral, dfdat,
            which_features = WHICH_FEATURE,
            trial_summary_score_ver="clust_sim_max")
plt.close("all")

##################### PLOTS
Cl = ClustDict[WHICH_FEATURE]
list_shape_basis = ParamsDict[WHICH_FEATURE]["list_shape_basis"]
list_strok_basis = ParamsDict[WHICH_FEATURE]["list_strok_basis"]

#### QUICK SMALL PLOTS
# yvar = "clust_sim_max"
savedir = f"{SDIR}/sim_score_histograms"
os.makedirs(savedir, exist_ok=True)        
list_yvar = ["sims_max", "sims_concentration", "sims_concentration_v2", "sims_entropy"]
ncols = 2
nrows = int(np.ceil(len(list_yvar)/ncols))
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*5, nrows*3))
for yvar, ax in zip(list_yvar, axes.flatten()):
    vals = Cl.cluster_extract_data("max_sim")[yvar]
    if yvar=="sims_entropy":
        ax.hist(vals, bins=50);
    else:
        ax.hist(vals, bins=np.linspace(0., max(vals), num=50));
#     ax.hist(DS.Dat[yvar], bins=np.linspace(0., 1., num=50));
    ax.set_xlabel(yvar)
path = f"{savedir}/hist-{yvar}.pdf"
savefig(fig, path)
plt.close("all")
    
### [Good] main plots
##### Plots of heatmaps, raw results, for feature spaces and clustering 
DS.clustergood_plot_raw_results(ClustDict, ParamsDict, ParamsGeneral, dfdat, SDIR)
plt.close("all")

##### Figures previously in characters
# Plot results for characters
from pythonlib.dataset.dataset_analy.characters import plot_clustering, plot_learning_and_characters, plot_prim_sequences
plot_clustering(DS, list_strok_basis, list_shape_basis, SDIR)
plt.close("all")

# Get indices spanning range of vals
savedir = f"{SDIR}/example_trials/{WHICH_FEATURE}"
os.makedirs(savedir, exist_ok=True)

valname = "clust_sim_max"
vals = DS.Dat[valname].tolist()
nplot = 40
from pythonlib.tools.listtools import random_inds_uniformly_distributed
indsplot = random_inds_uniformly_distributed(vals, nplot, return_original_values=False)
for ind in indsplot:
    val = vals[ind]
    prefix = f"{valname}-{val:.2f}"
    DS.clustergood_plot_single_dat(ind, savedir=savedir, prefix=prefix)
    plt.close("all")    


In [ ]:

#### DIM REDUCTIONS
savedir = f"{SDIR}/dim_reduction"
os.makedirs(savedir, exist_ok=True)

gmm_n_mixtures = range(4, 20)
things_to_do = ("tsne", "gmm", "gmm_using_tsne")
perplist = [15]
gmm_tsne_perp_to_use = 15
Cl.cluster_compute_all(gmm_n_mixtures=gmm_n_mixtures, perplist=perplist, 
                       things_to_do=things_to_do, 
                       gmm_tsne_perp_to_use=gmm_tsne_perp_to_use)

# Note: these corrs means that should (i) get better features and (ii) do PCA first before clsutering. 
simmat = Cl.Xinput
simmat_rownorm = simmat/np.sum(simmat, axis=1, keepdims=True)
cc = np.corrcoef(simmat_rownorm.T)
fig, _, _, _, _ = Cl._plot_heatmap_data(cc, labels_row=Cl.LabelsCols, labels_col=Cl.LabelsCols);
savefig(fig, f"{savedir}/xcorr_of_simmat.pdf")

##### PCA of sim mat

# Plot basis set
if False:
    # THis is plotted elsewhere
    shapes = Params["list_shape_basis"]
    # shapes = RES["list_shape_basis"]
    labels_col = Cl.LabelsCols
    assert shapes == labels_col
    strokes = Params["list_strok_basis"]
    fig, axes = DS.plot_multiple_strok(strokes, overlay=False, titles = shapes, ncols = len(shapes))
#     fig.savefig(f")

Cl.cluster_pca_plot_all(savedir=savedir)


for ver in ["gmm", "gmm_using_tsne"]:
    gmm_n_best, list_n, list_crossval, list_bic = Cl.cluster_gmm_extract_best_n(ver=ver)

    fig, axes = plt.subplots(2,2)

    ax = axes.flatten()[0]
    ax.plot(list_n, list_crossval, "-ok")
    ax.set_ylabel("crossval")
    ax.set_xlabel("gmm_n")

    ax = axes.flatten()[1]
    ax.plot(list_n, list_bic, "-ok")
    ax.set_ylabel("bic")
    ax.set_xlabel("gmm_n")

    savefig(fig, f"{savedir}/gmm_scores_using-ver_{ver}.pdf")

if False:
    # Plot, label by "alignsim" shapes (not useful for characters)
    list_perp = Cl.cluster_tsne_extract_list_perp()
    for perp in list_perp:
        Cl.cluster_plot_scatter("tsne", perp=perp, dims=[0, 1])    

# Plot in tsne space, using gmm labels
gmm_n_best, list_n, list_crossval, list_bic = Cl.cluster_gmm_extract_best_n()
for space in ["tsne", "pca"]:
    for label in ["shape", None, "col_max_sim", "gmm", "gmm_using_tsne"]:
        fig, axes = Cl.cluster_plot_scatter(space, label=label, gmm_n=gmm_n_best, perp=15, dims=[0, 1])
        savefig(fig, f"{savedir}/scatter-space_{space}-label_{label}.pdf")

### SAVE
import pickle
path = f"{SDIR}/DS.pkl"
with open(path, "wb") as f:
    pickle.dump(DS, f)
print("Saved to: ", path)

plt.close("all")

##### Finalizing "lumping" into distinct shape categories

In [ ]:
# PCA (which basis strokes are lumped together?)
Cl.cluster_pca_plot_all(savedir=savedir)

  

In [ ]:
pcamodel = Cl.ClusterComputeAllResults["pca_model"]
pcamodel.explained_variance_ # decresaeing
pcamodel.components_ # components_ndarray of shape (n_components, n_features), sorted by decreasing explained var.

In [ ]:
WHICH_FEATURE = "beh_motor_sim" 
Cl = ClustDict[WHICH_FEATURE]
list_shape_basis = ParamsDict[WHICH_FEATURE]["list_shape_basis"]
list_strok_basis = ParamsDict[WHICH_FEATURE]["list_strok_basis"]


In [ ]:
# Do PCA after first z-scor



In [ ]:
# Make a PCA reuslts cluster class
from pythonlib.cluster.clustclass import Clusters
pc_num_decreasing = list(range(pcamodel.components_.shape[0]))
feature_shapes = Cl.LabelsCols
assert list_shape_basis==feature_shapes
ClPCA = Clusters(pcamodel.components_, pc_num_decreasing, feature_shapes, ver="pca", params={"pcs_explained_var":pcamodel.explained_variance_})




In [ ]:
Xsorted, labelsSorted = ClPCA.sort_by_labels(labels=ClPCA.Params["pcs_explained_var"])
# ClPCA.plot_heatmap_data(sort_rows_by=ClPCA.Params["pcs_explained_var"], diverge=True);
ClPCA.plot_heatmap_data(diverge=True, ylabel="pc num (0=max var)");


In [ ]:
Cl.plot_save_hier_clust()

In [ ]:
DS.Dat.iloc[182]

In [ ]:
DS.Dat["clust_sim_max"]
DS.plot_multiple_after_slicing_within_range_values("clust_sim_max", 1, 1.2)

#### LUMPING sahpes into shape_label BY HAND

In [ ]:
# Keep only if score about this
threshold_circle = 0.9
threshold_all = 1.2

a = (DS.Dat["clust_sim_max_colname"]=="circle-6-1-0") & (DS.Dat["clust_sim_max"]>threshold_circle)
b = (DS.Dat["clust_sim_max_colname"]!="circle-6-1-0") & (DS.Dat["clust_sim_max"]>threshold_all)

In [ ]:
# Prune DS to keep just the good ones
DSpruned = DS.copy()
DSpruned.Dat = DSpruned.Dat[a | b].reset_index(drop=True)


In [ ]:
# Lump together (done by hand)
map_shapelump_to_shapes = {}
map_shape_to_shapelump = {}
for grp in [
    ["V-2-4-0", "arcdeep-4-4-0"],
    ["arcdeep-4-1-0", "V-2-1-0"]]:
    
    assert animal=="Pancho", "tjhis is hard coded for him"
    # name it after the first
    name = f"L|{grp[0]}"
    map_shapelump_to_shapes[name] = grp
    
    for g in grp:
        map_shape_to_shapelump[g] = name
        
list_shapes_final = []
for ind in range(len(DSpruned.Dat)):
    shape = DSpruned.Dat.iloc[ind]["clust_sim_max_colname"]
    if shape in map_shape_to_shapelump.keys():
        list_shapes_final.append(map_shape_to_shapelump[shape])
    else:        
        list_shapes_final.append(shape)
DSpruned.Dat["shape_label"] = list_shapes_final

In [ ]:
DSpruned.Dat["shape_label"].value_counts()

##### Give it other labels ("motor" level)


In [ ]:
# initial angle
DSpruned.features_compute_velocity_binned()

In [ ]:
# Location
DSpruned.Dat[:2]

#### Save this DS, so can later load into constructing Snippets [e.g., RSA plots]

In [ ]:
params = {
    "NOTEBOOK":"230623_STROKES_CLUSTERING_SIMILARITY",
    "WHICH_LEVEL":WHICH_LEVEL,
    "WHICH_BASIS_SET":WHICH_BASIS_SET,
    "ParamsGeneral":ParamsGeneral,
    "ParamsDict":ParamsDict,
    "WHICH_FEATURE":WHICH_FEATURE,
    "threshold_circle":threshold_circle,
    "threshold_all":threshold_all,
    "map_shapelump_to_shapes":map_shapelump_to_shapes,
    "map_shape_to_shapelump":map_shape_to_shapelump}


In [ ]:
DIR = f"/gorilla1/analyses/recordings/main/EXPORTED_BEH_DATA/DS/{animal}/{date}"
os.makedirs(DIR, exist_ok=True)
DSpruned.export_dat(DIR, params)



##### Loading presaved cluster labelsb

In [ ]:
# IGNORE, Stuff used in development. this is all now done automaticlaly in preprocessing..

In [ ]:
# [IGNORE] Try cleaning up dataset (buygs?)
D._cleanup_preprocess_each_time_load_dataset()
D.tokens_generate_replacement_from_raw()

# Wrapper to exatract and replace columns in D. 
D.charclust_shape_labels_extract_presaved_from_DS()

# 1) Replace all tokens with extracted shapes.
D.tokens_generate_replacement_from_raw()

# 2) delete older tokens
D.sequence_tokens_clear_behclass_and_taskclass_and_lock()


In [ ]:
### [MAIN STUFF that would break due to labels]

# 3. 
print(D.TokensVersion)

# 4) get sequence context stuff
D.seqcontext_preprocess()
D.seqcontext_plot_examples_and_print_context(10)

# Simulating what would do when load Snippets..
Dcopy = D.copy()

Dcopy.seqcontext_preprocess()

In [ ]:

# [EXTRACT DS]
from pythonlib.dataset.dataset_strokes import preprocess_dataset_to_datstrokes
DS = preprocess_dataset_to_datstrokes(D, "all_no_clean")
# DS = preprocess_dataset_to_datstrokes(D, "clean_chars_clusters_without_reloading")
# DS = preprocess_dataset_to_datstrokes(D, "clean_chars")
# clean_chars_clusters_without_reloading
DS.plotcheck_compare_to_dataset(1)

In [ ]:
DS.Dat["shape"].value_counts()

In [ ]:
DS.plotshape_singleshape_egstrokes_overlaid(shape="arcdeep-85-3.8-5")

In [ ]:
inds = DS.Dat[DS.Dat["shape"] == "Lcentered-86-2.4-5"].index.tolist()



In [ ]:
# DS.plot_single_overlay_entire_trial(104, overlay_beh_or_task="task")
# DS.plot_single_overlay_entire_trial(119, overlay_beh_or_task="task")
DS.plot_multiple_overlay_entire_trial(inds[:10], overlay_beh_or_task="task")

In [ ]:
# Rename them by comparing to canonical images


##### plot examples of all the prims (shapes, and example trials)

In [ ]:
%matplotlib inline
DS.plotshape_multshapes_egstrokes(ver_behtask="beh");

##### Debugging - loading of saved shape clusters in DS

In [ ]:
from pythonlib.dataset.dataset_strokes import preprocess_dataset_to_datstrokes
DS = preprocess_dataset_to_datstrokes(D, "all_no_clean")


In [ ]:
DS.clustergood_load_saved_cluster_shape_classes()

In [ ]:
D.Dat[~(D.Dat["task_kind"]=="character")]["seqc_0_shape"].value_counts()

In [ ]:
DS.Dat[DS.Dat["task_kind"]=="character"]["shape"].value_counts()

In [ ]:
DS.Dat["shape"].value_counts()

In [ ]:
DS.Dat[:10]

In [ ]:
import pickle
path = '/gorilla1/analyses/recordings/main/EXPORTED_BEH_DATA/DS/Diego/231204/DS_data.pkl'
with open(path, "rb") as f:
    df = pickle.load(f)

In [ ]:
df["shape_label"].value_counts()

##### Test out extraction of Snippets dataset for char trials

In [ ]:
from neuralmonkey.metadat.analy.anova_params import dataset_apply_params
D, DS, params = dataset_apply_params(D, None, "chartrial", animal, date) # prune it


In [ ]:
D.plotSingleTrial(40)

In [ ]:
Tk1 = list(D.TokensTask.values())[0]               
Tk2 = list(D.TokensTask.values())[0]
Tk1==Tk2

#### Save matrix of pairwise diatnces between strokes (e.g., for RSA analysis later).

In [ ]:
##### Should generate the distmat a single time, using batch function.
strokes = [S() for S in DS.Dat["Stroke"].tolist()]
distance_ver = "dtw_vels_2d"
similarity_matrix = DS._cluster_compute_sim_matrix_with_good_params(strokes, strokes, distance_ver) 


In [ ]:
# save trialcodes and stroke indices 
trialcodes = DS.Dat["trialcode"].tolist()
stroke_indices = DS.Dat["stroke_index"].tolist()


In [ ]:
# Save this in a permanent location
DIR = f"/gorilla1/analyses/recordings/main/EXPORTED_BEH_DATA/DS/{animal}/{date}/distance_matrix_all_strokes_pairwise/{distance_ver}"
os.makedirs(DIR, exist_ok=True)
path = f"{DIR}/data.npy"
with open(path, "wb") as f:
    np.save(f, similarity_matrix)
path = f"{DIR}/trialcodes_in_order.npy"
with open(path, "wb") as f:
    np.save(f, trialcodes)
    
path = f"{DIR}/stroke_indices_in_order.npy"
with open(path, "wb") as f:
    np.save(f, stroke_indices)
    
path = f"{DIR}/params.yaml"
params = {
    "distance_ver":distance_ver,
    "trialcodes_in_order":trialcodes, 
    "stroke_indices_in_order":stroke_indices
}

from pythonlib.tools.expttools import writeDictToYaml
writeDictToYaml(params, path)



In [ ]:
# TODO: 
# Plot the cases that have score less than threshold.
# - do they require a different basis set?

### DEVELOPING DISTANCE METRIC (low-level plots, to plot examples)


##### Good - pick one trial (stroke), and compare it to entire basis set

In [ ]:
import random
ind = random.sample(range(len(DS.Dat)), 1)[0]
print(ind) 

In [ ]:
# Given a distance metric, quickly use it scored against beh, and plot results,
# for a single trial

dfbasis, _, _ = DS.stroke_shape_cluster_database_load_helper(
    which_basis_set=None,
    which_shapes=None)
list_strok_basis = dfbasis["strok"].tolist()
list_shape_basis = dfbasis["shape"].tolist()

# for distancever in ["euclidian_diffs", "euclidian", "hausdorff_alignedonset", "dtw_vels_1d", "dtw_vels_2d"]:
for distancever in ["euclidian_diffs", "dtw_vels_2d"]:
    # distancever = "dtw_vels_2d"
    
    ###############
    strokes_data = [DS.Dat.iloc[ind]["strok"]]    
    cl = DS._cluster_compute_sim_matrix(strokes_data, list_strok_basis, distancever=distancever,
                                   return_as_Clusters=True)
    scores = cl.Xinput[0,:]
    DS._clustergood_plot_single_dat(ind, scores, list_strok_basis, list_shape_basis, YLIM=None)

##### Same, but pick out a specific few basis strokes, and plot deeper DEBUG plots

In [ ]:
# Pick out specific task stroke to compare to:
# Given a distance metric, quickly use it scored against beh, and plot results,
# for a single trial

inds_basis = [2, 13]
DEBUG=True
distancever = "dtw_vels_2d"
    
###############
strokes_data = [DS.Dat.iloc[ind]["strok"]]

dfbasis, _, _ = DS.stroke_shape_cluster_database_load_helper(
    which_basis_set=None,
    which_shapes=None)
list_strok_basis = dfbasis["strok"].tolist()
list_shape_basis = dfbasis["shape"].tolist()

list_strok_basis = [list_strok_basis[i] for i in inds_basis]
list_shape_basis = [list_shape_basis[i] for i in inds_basis]

cl = DS._cluster_compute_sim_matrix(strokes_data, list_strok_basis, distancever=distancever,
                               return_as_Clusters=True, DEBUG=DEBUG)

scores = cl.Xinput[0,:]
DS._clustergood_plot_single_dat(ind, scores, list_strok_basis, list_shape_basis, YLIM=None)

### Devo, towards good char - neural analysis

In [ ]:
### MAIN TODO:
# Use DTW - optimize it.

# goal: Different distance matrices based on different sets of features (hypotheses).
# - Motor spaces
# - Abstract spaces: First give it a category, then use that category.


In [ ]:
# 1) For each feature space, assign each trial it's best fitting value.

# 2) Prune data to just cases with good fit.
ClustDict


In [ ]:
# TODO (in _features_wrapper_generate_all_features):
# Give each row a tuple label --> allows plotting using sortings by the different labels.


In [ ]:
# For each trial, extracting clustering score, etc.
assert False, --> extract lsit of tuples that can assign to each of the Cl objects, i.e, not just that single which_feagures.

In [ ]:
####

##### OTHER STUFF

In [ ]:
# IN PROGRESS...
plot_learning_and_characters(D, SDIR)

In [ ]:
# ##### Plot histogram of clust sim scores, agg across all data
# 
# savedir = f"{SDIR}/sim_score_histograms"
# os.makedirs(savedir, exist_ok=True)
# # Compare different metrics for concentration...
# x1 = DS.Dat["clust_sim_concentration_v2"]
# x2 = simmat_entropy
# fig,ax = plt.subplots()
# ax.plot(x1, x2, "xk")
# WHICH_FEATURE = "beh_motor_sim"
# savedir = f"{SDIR}/example_trials/{WHICH_FEATURE}"
# os.makedirs(savedir, exist_ok=True)

### [Prep for Kavli talk] TODO

In [ ]:
# 1) Given set of trials, get score relative to different kinds of basis sets.
# 2) Visaulize/score how clustered it is --> e.g, what fraction trials are well-assigned, which are not?
# 3) Split trials based on image

# Then redo using different similarity metrics --> (i) image, (ii) motor (iii) combined (as it is now). 
# Then compare to null model.




##### Older code that doesnt work, but is obsolete (folded into above already)

In [ ]:

from pythonlib.dataset.dataset_analy.characters import debug_eyeball_distance_metric_goodness

debug_eyeball_distance_metric_goodness(D)

### Older 

In [ ]:

# Plot
RES = {
    "Cl":Cl,
    "DS":DS
}
# Cl = RES["Cl"]
# DS = RES["DS"]

DS.plot_single_overlay_entire_trial(ind, overlay_beh_or_task="beh");
DS.plot_single_overlay_entire_trial(ind, overlay_beh_or_task="task");

# Plot basis strokes
titles = [f"{x:.4}" for x in cl.Xinput[0,:]]
DS.plot_multiple_strok(list_strok_basis, ncols=10, overlay=False, titles=titles);

fig, ax = plt.subplots()
# plot line plot 

In [ ]:
DS.plot_single_overlay_entire_trial([ind], overlay_beh_or_task="task")


In [ ]:
# Given a feature, plot example trials, using the abstracted features
which_feature = "task_shape_cat_abstract"
Cl = DS.Clusters_ClustDict[which_feature]

RES = {
    "Cl":Cl,
    "DS":DS
}
# Cl = RES["Cl"]
# DS = RES["DS"]

##### DEBUGGING
from pythonlib.dataset.dataset_analy.characters import plot_example_trials
plot_example_trials(RES)

In [ ]:
##### Todo (clustering)

# pull out casees that dont score well against any prim
# use average prim (not just single example
# include direction variations?
# shapes are also define by reflection
# improve strok dist using both euclidian and euclidiandiff`b?


### MORE PLOTS

##### in progress below

In [ ]:
##### Which are the "most informative" tasks?

# for each datapt, compute its sim vector for each kind of representation. 
# Informative means you have diff similiarity vectors.
assert False, "replace D below, or else will delete dataset"

from scipy.spatial.distance import pdist, squareform

for key, Cl in ClustDict.items():
    D = squareform(pdist(Cl.Xinput))
    Cl.DistanceMatrices = {}
    Cl.DistanceMatrices["eucl"] = D
    


# For each row, compute the pairwise correlations between row vectors across all pairs of representations.
numrows = len(dfdat)
list_rep = ClustDict.keys()
list_shape = dfdat["shape"].values.tolist()

mat_r = []
mat_meandist = []
list_rep = ["beh_motor_sim", "task_image_sim"]
for i in range(numrows):
    
    # Raw feature vectors
    
    
    list_vec = []
    for rep in list_rep:
        Cl = ClustDict[rep]
        
        assert Cl.Labels == list_shape
        # get row vector, ignoring the diagnoanl
        vec = np.r_[Cl.DistanceMatrices["eucl"][i, :i], Cl.DistanceMatrices["eucl"][i, i+1:]]
        list_vec.append(vec)
    
    # compute corr for each pair of vec
    from scipy.stats import pearsonr
    list_r = []
    for i in range(len(list_vec)):
        for ii in range(len(list_vec)):
            if ii>i:
                r, _ = pearsonr(list_vec[i], list_vec[ii])
                list_r.append(r)
    mat_r.append(list_r)
    
    # mean distance to others
    list_mean_dist = []
    for vec in list_vec:
        mean_dist = np.mean(vec)
        list_mean_dist.append(mean_dist)
    mat_meandist.append(list_mean_dist)
    
    # 
    
    
mat_r = np.array(mat_r)
mat_meandist = np.array(mat_meandist)


# Plot
fig, axes = plt.subplots(2,2, figsize=(10, 14))
    
ax = axes.flatten()[0]
# ax.plot(list_shape, np.min(mat_r, axis=1))
ax.plot(np.min(mat_r, axis=1), list_shape, "or", label="min")
ax.plot(np.mean(mat_r, axis=1), list_shape, "ok", label="mean")
ax.grid()
ax.axvline(0)
ax.set_title("r")
ax.legend()
    
ax = axes.flatten()[1]
# ax.plot(list_shape, np.min(mat_r, axis=1))
ax.plot(np.mean(mat_meandist, axis=1), list_shape, "ok", label="mean")
ax.plot(np.min(mat_meandist, axis=1), list_shape, "or", label="min")
ax.plot(np.max(mat_meandist, axis=1), list_shape, "ob", label="max")
ax.grid()
# ax.axvline(0)
ax.set_title("distance")
ax.legend()
    
ax = axes.flatten()[2]
# ax.plot(list_shape, np.min(mat_r, axis=1))
for i in range(len(list_rep)):
    ax.plot(mat_meandist[:,i], list_shape, "o", label=list_rep[i])
ax.grid()
# ax.axvline(0)
ax.set_title("distance")
ax.legend()
    


In [ ]:
##### Find pairs which are (same stim, diff beh) or (same beh, diff stim)

# Plot the sample sizes for each shape
for shape in list_shape_reordered:
    n = sum(DS.Dat["shape_oriented"]==shape)
    print(shape, ' ----------- ', n)

# Compare beh and task distances
list_shape = dfdat["shape"].values.tolist()
Cl_beh = ClustDict["beh_motor_sim"]
Cl_task = ClustDict["task_image_sim"]
assert Cl_beh.Labels==list_shape
assert Cl_task.Labels==list_shape
# - iterate over all pairs of shapes
dat = []
for i, shape1 in enumerate(list_shape):
    for ii, shape2 in enumerate(list_shape):
        if ii<=i:
            continue
                
        sim_beh = Cl_beh.find_dat_by_label(shape1, shape2)
        sim_task = Cl_task.find_dat_by_label(shape1, shape2)
        
        dat.append({
            "ind_shape1":i,
            "ind_shape2":ii,
            "ind_shape_pair":f"{i}-{ii}",
            "shape1":shape1,
            "shape2":shape2,
            "sim_beh":sim_beh,
            "sim_task":sim_task
        })
        
        print(shape1, shape2, sim_beh, sim_task)

df = pd.DataFrame(dat)
df["sim_abs_diff"] = np.abs(df["sim_beh"] - df["sim_task"])
df["sim_beh_minus_task"] = df["sim_beh"] - df["sim_task"]


# dfthis = df[df["sim_beh_minus_task"]>0.5].reset_index(drop=True)
dfthis = df[df["sim_beh_minus_task"]<-0.5].reset_index(drop=True)
dfthis

sns.pairplot(data=dfthis, vars=["sim_beh", "sim_task"])

sns.scatterplot(data=dfthis, x="ind_shape_pair", y="sim_abs_diff")
sns.scatterplot(data=dfthis, x="ind_shape_pair", y="sim_abs_diff")

# for each pair, extract task and beh and plot

ind = 7

sh1 = dfthis.iloc[ind]["shape1"]
sh2 = dfthis.iloc[ind]["shape2"]
i1 = dfthis.iloc[ind]["ind_shape1"]
i2 = dfthis.iloc[ind]["ind_shape2"]
sim_beh = dfthis.iloc[ind]["sim_beh"]
sim_task = dfthis.iloc[ind]["sim_task"]

strok_beh_1 = dfdat.iloc[i1]["strok"]
strok_beh_2 = dfdat.iloc[i2]["strok"]
strok_task_1 = dfdat.iloc[i1]["strok_task"]
strok_task_2 = dfdat.iloc[i2]["strok_task"]

list_stroke_this = [strok_beh_1, strok_beh_2, strok_task_1, strok_task_2]
DS.plot_multiple_strok(list_stroke_this, overlay=False, ncols=2)

print("shapes:", sh1, " | ", sh2)
print("sim_beh:", sim_beh)
print("sim_task:", sim_task)

In [ ]:
# TODO:
1. Replace this with clustergood_featurespace_project
ClustDict = DS.features_generate_clusters_from_dataset(dfdat)
2. Conmbine all plots
3. Add features (e.g., starting stroke, etc)


### OLD PRIM PLOTS

### CHARACTERS

In [ ]:
##### Generate data
from pythonlib.dataset.dataset_analy.characters import pipeline_generate_and_plot_all, generate_data

In [ ]:
do_plots = True
RES, savedir = pipeline_generate_and_plot_all(D, do_plots, filter_taskkind=None)

if False:
    filter_taskkind = None

    # list_distance_ver=["hausdorff_alignedonset"]
    # list_distance_ver=["euclidian_diffs", "euclidian"]
    # list_distance_ver=["euclidian"]
    list_distance_ver = None

    # RES, savedir = pipeline_generate_and_plot_all(D, False, filter_taskkind=filter_taskkind,
    #                                              list_distance_ver=list_distance_ver)

    ds_filterdict = {"gridsize":["rig3_3x3_small"]}
    ds_clean_methods = ["remove_if_multiple_behstrokes_per_taskstroke", "stroke_too_short"]
    ds_clean_params = {
        "min_stroke_length":50
    }

    RES = generate_data(D, list_distance_ver=list_distance_ver, filter_taskkind=filter_taskkind,
                 ds_clean_methods=ds_clean_methods, ds_clean_params=ds_clean_params,
                 ds_filterdict=ds_filterdict)

##### Plot exmaple strokes, sorted by labels

In [ ]:
# 1) give dummy columnname in DS, based on label

gmm_n = 11
# gmm_n = 17

labels = Cl.cluster_extract_label("gmm_using_tsne", gmm_n)
DS.Dat["labels_dummy"] = labels
fig = DS.plotshape_multshapes_trials_grid("labels_dummy", nrows=4)

### PLOTS originally made for single prims

In [ ]:
SDIR = D.make_savedir_for_analysis_figures_BETTER("clustering_prims_new")

In [ ]:
DS.featu

In [ ]:
##### Distributions of combos of shapes used in each char


# Which shapes tend to be drawn correctly?

from pythonlib.dataset.dataset_analy.characters import plot_prim_sequences
plot_prim_sequences(RES, D, savedir)


# 2) Cluster stroke, along different dimensions --> plot ordered by cluster.

# 3) For each unique character, plot it and the strokes used for it



# For each character, get each stroke's clustering score.


# 3) For each unique character, plot it and the strokes used for it





### Compare feature spaces

e.g., theymake different predictions for similarity space in neural activity

In [ ]:
#

## PUTTING ALL TOGETHER

In [ ]:
use both trials and shape average.


### Getting random parses from primitives

In [ ]:
doesnt make sense...


### TESTING - getting random parses

In [ ]:
import numpy as np
# import torch
import matplotlib.pyplot as plt
from pythonlib.drawmodel.parsing import get_parses_from_strokes

In [ ]:
USE_TORCH = False
# VERY FAST
params_parse = {
    "configs_per":5,
    "trials_per":10,
    "max_ntrials":25, 
    "max_nwalk":25,
    "max_nstroke":25
}
return_in_strokes_coords = True
kparses = 10
# animal = "Red"

use_extra_junctions=True
# score_ver = "travel"
score_ver = "travel_from_orig" # better, since differentiates 2 tasks thjat are just flipped (and so will not throw one of them out)
score_norm = "negative"
image_WH = 105


In [ ]:
# Get sketchpad edges
maxes = []
for k, v in D.Metadats.items():
    maxes.append(np.max(np.abs(v["sketchpad_edges"].flatten())))
canvas_max_WH = np.max(maxes)

# origin for tasks, is generally around (0, 50) to (0,100). so just hard code here.
# origin = torch.tensor([image_WH/2, -(0.45*image_WH)])
origin = np.array([image_WH/2, -(0.45*image_WH)])
# alternatively, could look into D.Dat["origin"], and convert to image coords [(1,105), (-105, -1)]

# For each row, parse its task
score_fn = lambda parses: score_function(parses, ver=score_ver, 
                                         normalization=score_norm, use_torch=USE_TORCH, origin=origin)

In [ ]:
if False:
    # Just testing, pick a random trial
    import random
    ind = random.sample(range(len(D.Dat)), 1)[0]
    strokes = D.Dat["strokes_task"].values[ind]

    if False:
        from pythonlib.tools.stroketools import strokesInterpolate2
        strokes = strokesInterpolate2(strokes, N=["npts", 100])
    else:
        pass
    parses, log_probs_k = get_parses_from_strokes(strokes, canvas_max_WH, 
                                                  use_extra_junctions=use_extra_junctions, plot=True,
                                                 return_in_strokes_coords=True, k=5)

# save params
params_parse["canvas_max_WH"] = canvas_max_WH
params_parse["use_extra_junctions"] = use_extra_junctions
params_parse["return_in_strokes_coords"] = return_in_strokes_coords
params_parse["score_ver"] = score_ver
params_parse["score_norm"] = score_norm

# OLD VERSION - GET EACH ROW
#     # Collect parses
#     PARSES = []
#     for row in D.Dat.iterrows():
#         if row[0]%100==0:
#             print(row[0])
#         strokes = row[1]["strokes_task"]
#         index = row[0]
#         trial_id = row[1]["trialcode"]
#         unique_task_name = row[1]["unique_task_name"]
#         character = row[1]["character"]

#         parses, log_probs = get_parses_from_strokes(strokes, canvas_max_WH, 
#                                                   use_extra_junctions=use_extra_junctions, plot=False,
#                                                  return_in_strokes_coords=return_in_strokes_coords, k=k,
#                                                    configs_per = params_parse["configs_per"],
#                                                    trials_per = params_parse["trials_per"],
#                                                    max_ntrials = params_parse["max_ntrials"],
#                                                    max_nstroke = params_parse["max_nstroke"],
#                                                    max_nwalk = params_parse["max_nwalk"],
#                                                    )

#         PARSES.append(
#             {"strokes_task":strokes,
#              "index_dat":index,
#              "trial_id":trial_id,
#              "unique_task_name":unique_task_name,
#              "character":character,
#              "parses":parses,
#              "parses_log_probs":log_probs}
#         )

# NEW VERSION - only do once for each unique task
tasklist = sorted(list(set(D.Dat["unique_task_name"])))
# Collect parses
PARSES = []

for i, task in enumerate(tasklist):
    # find the first row that has this task
    row = D.Dat[D.Dat["unique_task_name"]==task].iloc[0]
    assert row["unique_task_name"]==task

#         if i%20==0:
#             print(i, "-",  task)
    print(i, "-",  task)
    strokes = row["strokes_task"]

    parses, log_probs = get_parses_from_strokes(strokes, canvas_max_WH, 
                                              use_extra_junctions=use_extra_junctions, 
                                              score_fn=score_fn,
                                                plot=False, image_WH=image_WH,
                                                return_in_strokes_coords=return_in_strokes_coords, 
                                                k=kparses, configs_per = params_parse["configs_per"],
                                               trials_per = params_parse["trials_per"],
                                               max_ntrials = params_parse["max_ntrials"],
                                               max_nstroke = params_parse["max_nstroke"],
                                               max_nwalk = params_parse["max_nwalk"],
                                               )
    assert len(parses)>0, "why?"

In [ ]:
    PARSES.append(
        {"strokes_task":strokes,
         "unique_task_name":task,
         "parses":parses,
         "parses_log_probs":log_probs}
    )

# === save as dataframe
import os
import pandas as pd
import pickle

DIR, FNAME = os.path.split(paththis)
fname_parse = f"{DIR}/{FNAME}/parses.pkl"
print("Saving at:")
print(fname_parse)

PARSES  = pd.DataFrame(PARSES)
PARSES.to_pickle(fname_parse)

fname_params = f"{DIR}/{FNAME}/parses_params.pkl"
with open(fname_params, "wb") as f:
    pickle.dump(params_parse, f)